##Databases: datasette

1. Introduction

Datasette - an easier way to view a file (ex; csv) by creating a view on a server. Assumes data lives in a sqlite database.
  
    -Take a csv file -> turn into a sqlite database -> open with datasette.

In a venv on terminal, type below lines. This will give you a link to open up your server through a 'localhost' type url. Also very easy to share the server with others.
   
    -csvs-to-sqlite data.csv data.db
    
    -datasette data.db

Once you opened the link you can run custom SQL queries, view the database, and simple filter query through a search bar mechanic. Very simple!

2. VISUALIZATION - Best for sharing results via your server as a service.

    -pip install datasette-vega

    -datasette data.db

This is an incredibly easy way to create bar, line, or scatterplots through the plugin. Click to make the bar charts, with the ability to still filter rows through search bar.


3. SQL

Facet on a column filter is almost like a select statement, you can view all the unique values in the row alongside the count of rows with that value. You can further drill down to create a subset of the df as well.

You can click view/edit SQL button to recieve the customized SQL statement to get to your exact view in the dataframe.

This server url is safe due to the fact that sqlite.db is hosted by datasette in a read only view. That way you can't make any updates or messy queries with it.

4. API

The interface can technically be viewed as an API because you can export custom query views as a JSON file or csv.

- csv file export link will give you an http link, can be opened and read through pandas! pd.read_csv('link')

- JSON file export link will also give an http but open it like this:
   

    -from clumper import Clumper
    
    -url = 'link' #The sql query will be encoded into the link
    
    -Clumper.read_json(url).show(1)




5. DEPLOY

You can use docker as a serverless service (free for usuage). datasette can create a docker and push it to a cloud service.


To publish datasette database


    -datasette publish cloudrun data.db -service=my_data
    -#Command may ask for your input in choosing region. It will provide a list of numeric values representing various regions in the world.


6. AUTH

You can add metadata like title, description, custom description for each database, source of data tags.

    -datasette data.db -m metadata.json

You can also use a datasette plugin to ask for authentication like a password


    -pip install datasette-auth-password

You will need to use a password hash (think encryption for a string). This could be important depending on the sensitivity of the database your working with.

7. DATA EXPLORATION

https://github.com/simonw/datasette

##STREAMLIT

1. HELLO WORLD


    -pip install streamlit

    -streamlit --help

    -streamlit hello (opens a local port)

2. SLIDERS



In [ ]:
import streamlit as st
import numpy as np

st.title["Micah's Site"] #streamlit will recognize this line as a title
st.write["This is Micah Richardson's website! Cool data science projects to come."]

Next in the terminal type  
    
    -streamlit run app01.py

A slider is a user interface element.

In [ ]:
x = st.slider('Slope', min_value=.01, max_value=.10, step =.01)
y = .slider('Noise', min_value=.01, max_value=.10, step =.01)

In [ ]:
st.write(f"x={x} y={y}")

The st.write allows for a dynamic print statement on defined variables basically

3. LINECHART

    -values = np.cumprod(1+np.random.normal(x,y,(100,10)),axis =0)
   
    -st.line_chart(values)

This code builds upon previous section, the x and y in the code make the line chart dymanic to what the slider values end up being.
Streamlit uses altair ->vega visualization packages

4. MATPOLOTLIB

    import matplotlib as plt

    for i in range(values.shape[1]):
       plt.plot(values[:,i])

st.pyplot()

5. SIDEBAR


    x = st.sidebar.slider('Slope', min_value=.01, max_value=.10, step =.01)
    
    y = st.sidebar.slider('Noise', min_value=.01, max_value=.10, step =.01)

    st.sidebar.markdown("You can now **change** the values to change the *chart*")

Note that *word* makes it italics, **word** makes it bold

6. AlGOROTHIM SETTINGS

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

    n = 1000
    np.random.seed(0,6,n)
    X = np.linspace(0,6,n)[:,np.newaxis]
    y = np.sin(X).ravel()+np.sin(6*X).ravel()+np.random(n) *.3

    n_est = st.sidebar.slider("n_est",min_value=1,max_value=5_000,step=1)
    mod1 = DecisionTreeRegressor(max_depth=4)
    y1 = mod1.fit(X,y).predict(X)
    y2 = AdaBoostRegressor(mod1,n_estimators=n_est).fit(X,y).predict(X)

    plt.scatter(x,y,alpha=.1)
    plt.plot(x,y1,label="just a decision tree")
    plt.plot(x,y2,label=f'adaboost-{n_est}")
    plt.legend()

    st.pyplot()

ADABOOSTREGRESSOR model - learns from the mistakes made before and trys to predict mistakes again



7. CHECKBOXES

    if st.sidebar.checkbox("Toggle Scatter Plot")
      plt.scatter(x,y,alpha=.1)

  The code above makes a checkbox that when clicked/unclicked re-runs the entire model and will add a scatterplot

8. CACHE


    @st.cache
Used above a function, this will cache the function such that the model results will be stored in a lookup table. This saves the machine time rerunning the model everytime a user makes an action. Best for pre-computing on large datasets.

9. PANDAS

  

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

st.title('BigMac Index')

@st.cache
def load_data():
    data = pd.read_csv("bigmac.csv")
    return data.assign(date = lambda d: pd.to_datetime(d['date']))

df = load_data()

countries = st.sidebar.multiselect( #output is a list of values)
    "Select Countries",
    df['name'].unique()
)

varname = st.sidebar.selectbox( #allows for two options of local price and dollar price in a drop down menu
    "Select Column",
    ("local_price", "dollar_price")
)

subset_df = df.loc[lambda d: d['name'].isin(countries)] #only take the rows where the name is in the d['name] list

for name in countries:
    plotset = subset_df.loc[lambda d: d['name'] == name]
    plt.plot(plotset['date'], plotset[varname], label=name) #select all the names in the list
plt.legend()
st.pyplot()

if st.sidebar.checkbox("Show Raw Data"):
    st.markdown("### Raw Data")
    st.write(subset_df)

